In [1]:
import glob
import os
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import sklearn.metrics as metrics
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.preprocessing.image import img_to_array,load_img

In [2]:
dir1="train_leaves\TR03\Myo"
img_list=glob.glob(dir1+"/*")
list_image_tr03=[]

for img in img_list:
    temp_img=load_img(img,target_size=(200,200))
    temp_img_array = img_to_array(temp_img) /255
    list_image_tr03.append(temp_img_array)
list_image_tr03=np.array(list_image_tr03)
list_image2=list_image_tr03.reshape(-1,40000)
df_tr03=pd.DataFrame(list_image2)
df_tr03['label'] = np.full(df_tr03.shape[0],2)

In [3]:
dir2="train_leaves\TR36\Myo"
img_list2=glob.glob(dir2+"/*")
list_image_tr36=[]

for img2 in img_list2:
    temp_img1=load_img(img2,target_size=(200,200))
    temp_img2_array = img_to_array(temp_img1) /255
    list_image_tr36.append(temp_img2_array)
list_image_tr36=np.array(list_image_tr36)
list_image2_tr36=list_image_tr36.reshape(-1,40000)
df_tr36=pd.DataFrame(list_image2_tr36)
df_tr36['label'] = np.full(df_tr36.shape[0],1)

In [4]:
Df = pd.concat([df_tr03, df_tr36], ignore_index=True)

In [5]:
from tensorflow.keras.utils import to_categorical
x_train, x_test, y_train, y_test = train_test_split(Df.iloc[:,0:-1], Df.iloc[:,-1], test_size=0.20, random_state=None)

X_train = x_train.values.reshape(-1,200,200,1)
X_test = x_test.values.reshape(-1,200,200,1)
Y_train = to_categorical(y_train)
Y_test = to_categorical(y_test)

In [6]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
from PIL import Image
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, Conv2D, MaxPool2D, Flatten
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from tensorflow.keras.layers import MaxPooling2D, GlobalAveragePooling2D
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Activation

In [7]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, BatchNormalization, Flatten, Dense, Add

def residual_block(x, filters, strides=1, downsample=False):
    y = Conv2D(filters=filters, kernel_size=3, strides=strides, padding='same')(x)
    y = BatchNormalization()(y)
    y = tf.nn.relu(y)
    y = Conv2D(filters=filters, kernel_size=3, padding='same')(y)
    y = BatchNormalization()(y)

    if downsample:
        x = Conv2D(filters=filters, kernel_size=1, strides=strides, padding='same')(x)
        x = BatchNormalization()(x)

    return tf.nn.relu(Add()([x, y]))

def build_resnet(input_shape, num_classes):
    input_layer = Input(shape=input_shape)

    # Initial Convolutional Layer
    x = Conv2D(filters=64, kernel_size=7, strides=2, padding='same')(input_layer)
    x = BatchNormalization()(x)
    x = tf.nn.relu(x)
    x = MaxPooling2D(pool_size=3, strides=2, padding='same')(x)

    # Residual Blocks
    x = residual_block(x, filters=64)
    x = residual_block(x, filters=64)
    x = residual_block(x, filters=128, strides=2, downsample=True)
    x = residual_block(x, filters=128)
    x = residual_block(x, filters=256, strides=2, downsample=True)
    x = residual_block(x, filters=256)
    x = residual_block(x, filters=512, strides=2, downsample=True)
    x = residual_block(x, filters=512)

    # Global Average Pooling
    x = tf.reduce_mean(x, axis=[1, 2])
    
    # Fully Connected Layer
    x = Flatten()(x)
    x = Dense(units=num_classes, activation='softmax')(x)

    model = Model(inputs=input_layer, outputs=x)
    return model

# Define input shape
input_shape = (200, 200, 1)
num_classes = 3  # Assuming you have 3 classes

# Build the model
resnet_model = build_resnet(input_shape, num_classes)

# Compile the model
resnet_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = resnet_model.fit(X_train, Y_train, batch_size=32, epochs=10, validation_data=(X_test, Y_test))





Epoch 1/10


6/6 [==============================] - 27s 2s/step - loss: 3.3652 - accuracy: 0.4444 - val_loss: 1.4465 - val_accuracy: 0.5625
Epoch 2/10
6/6 [==============================] - 9s 1s/step - loss: 0.9922 - accuracy: 0.6032 - val_loss: 9.5246 - val_accuracy: 0.5833
Epoch 3/10
6/6 [==============================] - 11s 2s/step - loss: 0.6052 - accuracy: 0.6772 - val_loss: 5.7558 - val_accuracy: 0.5833
Epoch 4/10
6/6 [==============================] - 16s 3s/step - loss: 0.5522 - accuracy: 0.7090 - val_loss: 4.9673 - val_accuracy: 0.5625
Epoch 5/10
6/6 [==============================] - 14s 2s/step - loss: 0.4238 - accuracy: 0.8095 - val_loss: 4.5652 - val_accuracy: 0.4375
Epoch 6/10
6/6 [==============================] - 10s 2s/step - loss: 0.4326 - accuracy: 0.8201 - val_loss: 4.7929 - val_accuracy: 0.5625
Epoch 7/10
6/6 [==============================] - 10s 2s/step - loss: 0.3466 - accuracy: 0.8519 - val_loss: 5.1891 - val_accuracy: 0.5625
Epoch 8/10
6/6 [==============

In [8]:
predictions = resnet_model.predict(X_test)

2/2 [==============================] - 3s 282ms/step


In [9]:
(eval_loss,eval_accuracy)=resnet_model.evaluate(X_test,Y_test,batch_size=32,verbose=1)
print("Accuracy :{:.2f}%".format(eval_accuracy*100))
print("Loss :{:.2f}%".format(eval_loss))

2/2 [==============================] - 3s 666ms/step - loss: 1.3327 - accuracy: 0.5625
Accuracy :56.25%
Loss :1.33%
